In [ ]:
# @title **Import Python modules**

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import plotly.graph_objects as go
import pandas as pd

1. Calculate covariance matrix and calculate the eigenvectors and eigenvalues .\
gmx covar -s md.gro -f mdfit.xtc -o eigenvalues.xvg -v eigenvectors.trr -xpma covapic.xpm
2. Calculate PC1 and PC2.\
gmx anaeig -f md.xtc -s md.gro -v eigenvectors.trr -last 1 -proj pc1.xvg
gmx anaeig -f md.xtc -s md.gro -v eigenvectors.trr -frist 2 -last 2 -proj pc2.xvg
3. Concatenate PC1 and PC2 in one file.\
paste pc1.xvg pc2.xvg  | awk '{print Dollarsign1, Dollarsign2, Dollarsign4}' > PC1PC2.xvg
4. Calculate Gibbs Free Energy.\
gmx sham -f PC1PC2.xvg -ls FES.xpm

In [ ]:
https://github.com/jobinjobzz/FEL_3D-map-generation-script/blob/main/xpm2txt.py

In [ ]:
#!/usr/bin/env python

import sys

"""
Utility tool to convert xpm files generated by GROMACS to a 3-column text file.
"""

USAGE = "USAGE: xpm2txt.py -f <input xpm file> -o <output txt file> [-s]\n"
USAGE+= "Options:\n"
USAGE+= "\t-s\t(int)\tSorts the output by a given column"
USAGE+= "\n" # always keep this line

# Parse arguments
read_input, read_output, sort = False, False, False
xpm_file, out_file, column_sort = None, None, None
for arg in sys.argv[1:]:
    if read_input:
        read_input = False
        xpm_file = arg
    elif read_output:
        read_output = False
        out_file = arg
    elif sort:
        sort = False
        column_sort = int(arg)
    if arg[0] == "-":
        if arg == "-f":
            read_input = True
            continue
        elif arg == "-o":
            read_output = True
            continue
        elif arg == "-s":
            sort = True
        else:
            print (USAGE)
            sys.stderr.write('ERROR: Option not recognized: %s\n' %arg)
            sys.exit(1)

if not xpm_file:
    print (USAGE)
    sys.stderr.write('ERROR: You forgot to provide an input file.\n')
    sys.exit(1)
if not out_file:
    out_file = "out.txt"

# Parse XPM file
xpm_handle = open(xpm_file)
xpm_data = []
x_axis, y_axis = [], []
letter_to_value = {}
for line in xpm_handle:
    if line.startswith("/* x-axis"):
        x_axis = map(float, line.split()[2:-2]) # We trim the last value

    if line.startswith("/* y-axis"):
        y_axis = map(float, line.split()[2:-2]) # We trim the last value

    if line.startswith('"') and x_axis and y_axis: # Read data
        xpm_data.insert(0, line.strip().strip(',')[1:-1])

    if line.startswith('"') and len(line.split()) > 4:
        letter = line.split()[0][1:]
        value = float(line.split()[-2][1:-1])
        letter_to_value[letter] = value
xpm_handle.close()

# Match x/y/data
txt_values = []
for y_index, data_value in enumerate(xpm_data):
    y_value = y_axis[y_index]
    for x_index, x_value in enumerate(x_axis):
        txt_values.append([x_value, y_value, letter_to_value[data_value[x_index]]])

# Apply sorting if requested
if column_sort:
    try:
        txt_values.sort(key=lambda x: x[column_sort-1])
    except IndexError:
        print (USAGE)
        sys.stderr.write('ERROR: Column not found (%s)\n' %(column_sort))
        sys.exit(1)

# Print to file
out_handle = open(out_file, 'w')
for x, y, z in txt_values:
    out_handle.write("%3.5f\t%3.5f\t%3.5f\n" %(x,y,z))
out_handle.close()



In [13]:
#@title **3D Free Energy Landscape** { run: "auto" }

# Assuming 'fel.txt' has three columns: X, Y, Z
FEL_input = "fel.txt" #@param {type:"string"}
data = np.loadtxt(FEL_input)

# Extracting X, Y, and Z
X = data[:, 0]
Y = data[:, 1]
Z = data[:, 2]
X_Axis_Lable = "PC1" #@param {type:"string"}
Y_Axis_Lable = "PC2" #@param {type:"string"}
Z_Axis_Lable = "Free Energy" #@param {type:"string"}
# Reshape Z to make it 2D
Z = Z.reshape((len(np.unique(X)), len(np.unique(Y))))

# Create grid for X and Y
X, Y = np.meshgrid(np.unique(X), np.unique(Y))

Color = "electric" #@param ["aggrnyl","agsunset","blackbody","bluered","blues","blugrn","bluyl","brwnyl","bugn","bupu","burg","burgyl","cividis","darkmint","electric","emrld","Viridis_r","hsv","icefire","phase","twilight","mrybm","mygbm"]

fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z, colorscale =Color)])
fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
fig.update_layout(title='Free Energy Landscape',
                  autosize=False,
                  scene_camera_eye=dict(x=1.87, y=0.88, z=-0.64),
                  width=1024, height=1024,
                  margin=dict(l=65, r=50, b=65, t=90),
                    scene=dict(
        xaxis_title=X_Axis_Lable,
        yaxis_title=Y_Axis_Lable,
        zaxis_title=Z_Axis_Lable,
    ),
)


fig.show()
